In [1]:
"""
main.py
Time-Series Forecasting Project: LSTM (multivariate) + Walk-forward CV + SHAP explainability
Generates synthetic multivariate data (3 interacting features), trains an LSTM for multi-step forecasts,
runs walk-forward validation, computes RMSE/MAE, and produces SHAP explainability outputs.
"""

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam
import shap
import joblib
import json

# -------------------------
# Configuration
# -------------------------
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

OUT_DIR = "outputs"
os.makedirs(OUT_DIR, exist_ok=True)

LOOKBACK = 60       # how many past timesteps used as input
HORIZON = 14        # multi-step forecast horizon
N_FEATURES = 3      # number of features (multivariate)
EPOCHS = 30
BATCH_SIZE = 32

# -------------------------
# 1) Synthetic multivariate dataset (3 interacting features)
# -------------------------
def generate_multivariate_series(n_days=1500):
    t = np.arange(n_days)
    # Base trend
    trend = 0.01 * t
    # Seasonal component (yearly-like) and weekly-like
    season_year = 5.0 * np.sin(2 * np.pi * t / 365.0)
    season_week = 2.0 * np.sin(2 * np.pi * t / 7.0)
    # Random noise
    noise = np.random.normal(0, 0.8, size=n_days)

    # Feature 1: main target signal (combination of trend + season + noise)
    y = 20 + trend + season_year + season_week + noise

    # Feature 2: interacting regressor correlated with monthly cycles and lagged influence
    reg1 = 3.0 * np.sin(2 * np.pi * t / 30.0) + 0.5 * np.roll(y, 7) * 0.01
    reg1 += np.random.normal(0, 0.3, n_days)

    # Feature 3: exogenous signal that sometimes spikes (simulates promotions / events)
    spikes = np.zeros(n_days)
    spike_indices = np.random.choice(np.arange(50, n_days-1, 50), size=int(n_days/300)+1, replace=False)
    spikes[spike_indices] = np.random.uniform(5, 12, size=len(spike_indices))
    reg2 = 1.5 * np.cos(2 * np.pi * t / 90.0) + spikes + np.random.normal(0, 0.4, n_days)

    df = pd.DataFrame({
        "ds": pd.date_range("2015-01-01", periods=n_days, freq="D"),
        "y": y,
        "reg1": reg1,
        "reg2": reg2
    })
    return df

df = generate_multivariate_series(n_days=1500)
df.to_csv(os.path.join(OUT_DIR, "synthetic_multivariate.csv"), index=False)
print("Data generated and saved to outputs/synthetic_multivariate.csv")

# -------------------------
# 2) Supervised framing: create sliding windows
# -------------------------
def create_supervised(df, lookback=LOOKBACK, horizon=HORIZON, features=["y","reg1","reg2"]):
    X, Y = [], []
    df_vals = df[features].values
    n = len(df_vals)
    for i in range(lookback, n - horizon + 1):
        past = df_vals[i-lookback:i, :]   # shape (lookback, n_features)
        future = df_vals[i:i+horizon, 0]  # target 'y' next horizon
        X.append(past)
        Y.append(future)
    X = np.array(X)  # (samples, lookback, features)
    Y = np.array(Y)  # (samples, horizon)
    return X, Y

features = ["y","reg1","reg2"]
X, Y = create_supervised(df, LOOKBACK, HORIZON, features=features)
print("Supervised shapes:", X.shape, Y.shape)

# -------------------------
# 3) Walk-forward cross-validation (rolling forecast origin)
# -------------------------
def build_lstm_model(lookback, n_features, horizon):
    model = Sequential([
        Input(shape=(lookback, n_features)),
        LSTM(64, return_sequences=False),
        Dropout(0.2),
        Dense(64, activation="relu"),
        Dense(horizon)   # multi-step output
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss="mse")
    return model

def walk_forward_cv(X, Y, n_splits=5, initial_train_fraction=0.5):
    n_samples = X.shape[0]
    initial_train = int(n_samples * initial_train_fraction)
    fold_size = (n_samples - initial_train) // n_splits
    metrics = []
    preds_all = []
    trues_all = []
    models = []

    for fold in range(n_splits):
        train_end = initial_train + fold * fold_size
        val_end = train_end + fold_size
        if val_end > n_samples:
            val_end = n_samples

        X_train = X[:train_end]
        Y_train = Y[:train_end]
        X_val = X[train_end:val_end]
        Y_val = Y[train_end:val_end]

        if len(X_val) == 0:
            break

        model = build_lstm_model(LOOKBACK, N_FEATURES, HORIZON)
        model.fit(X_train, Y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0)
        Y_pred = model.predict(X_val)
        # compute metrics per-fold (flatten over horizon)
        rmse = np.sqrt(mean_squared_error(Y_val.flatten(), Y_pred.flatten()))
        mae = mean_absolute_error(Y_val.flatten(), Y_pred.flatten())

        metrics.append({"fold": fold, "rmse": float(rmse), "mae": float(mae)})
        preds_all.append(Y_pred)
        trues_all.append(Y_val)
        models.append(model)
        print(f"Fold {fold} | train_end={train_end} | val_end={val_end} | RMSE={rmse:.4f} MAE={mae:.4f}")

    return metrics, preds_all, trues_all, models

print("Starting walk-forward CV (this may take a few minutes)...")
metrics, preds_all, trues_all, models = walk_forward_cv(X, Y, n_splits=4, initial_train_fraction=0.5)

# Save metrics
with open(os.path.join(OUT_DIR,"cv_metrics.json"), "w") as f:
    json.dump(metrics, f, indent=2)
print("Cross-validation metrics saved to outputs/cv_metrics.json")

# -------------------------
# 4) Select final model (last trained model) and do full training on all available training data
# -------------------------
final_model = build_lstm_model(LOOKBACK, N_FEATURES, HORIZON)
final_model.fit(X[:-HORIZON], Y[:-HORIZON], epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1)
final_model.save(os.path.join(OUT_DIR, "final_lstm_model.keras"))
joblib.dump({"config": {"LOOKBACK": LOOKBACK, "HORIZON": HORIZON, "features": features}}, os.path.join(OUT_DIR, "meta.joblib"))

# -------------------------
# 5) Forecast final horizon on last available window and visualize
# -------------------------
last_input = X[-1:]  # most recent lookback window
pred_final = final_model.predict(last_input)[0]  # shape (HORIZON,)
dates_future = pd.date_range(df["ds"].iloc[-1] + pd.Timedelta(days=1), periods=HORIZON, freq="D")
forecast_df = pd.DataFrame({"ds": dates_future, "yhat": pred_final})
forecast_df.to_csv(os.path.join(OUT_DIR, "final_forecast.csv"), index=False)
print("Final forecast saved to outputs/final_forecast.csv")

# Plot last test actual vs predicted (use last available val set)
# We'll compare the final predicted horizon vs actuals (if available in data)
actual_future = df["y"].iloc[-HORIZON:].values
plt.figure(figsize=(10,5))
plt.plot(dates_future, pred_final, label="Forecast (pred)")
plt.plot(df["ds"].iloc[-HORIZON:], actual_future, label="Actual (last HORIZON days)")
plt.legend()
plt.title("Final multi-step forecast vs actual (last available window)")
plt.savefig(os.path.join(OUT_DIR, "forecast_vs_actual.png"))
plt.close()

# -------------------------
# 6) SHAP explainability (DeepExplainer for Keras LSTM)
# -------------------------
# DeepExplainer requires a background sample; use a small subset of training data
background = X[:200] if X.shape[0] > 200 else X[:max(1, X.shape[0]//4)]
# Use the model's prediction function wrapper for shap
# For DeepExplainer, inputs must be a list matching model inputs; we have single 3D input
try:
    explainer = shap.DeepExplainer(final_model, background)
    # compute SHAP values for the last N samples used for explanation
    to_explain = X[-200:] if X.shape[0] >= 200 else X[-20:]
    shap_values = explainer.shap_values(to_explain)  # shap_values is a list (one per model output)
    # For multi-output (HORIZON outputs), shap_values is a list of arrays (each: samples, lookback, features)
    # We'll summarize by averaging across horizon outputs and across lookback timesteps for each feature
    # Convert to feature-level importances
    # shap_values_per_output -> list of arrays shape (samples, lookback, features)
    mean_abs_per_feature = None
    for out_idx, arr in enumerate(shap_values):
        # arr: (samples, lookback, features)
        # take mean absolute across samples and lookback -> (features,)
        abs_mean = np.mean(np.abs(arr), axis=(0,1))
        if mean_abs_per_feature is None:
            mean_abs_per_feature = abs_mean
        else:
            mean_abs_per_feature += abs_mean
    mean_abs_per_feature = mean_abs_per_feature / len(shap_values)
    # Save importance to JSON
    fi = {feat: float(val) for feat,val,feat in zip(mean_abs_per_feature, mean_abs_per_feature, features)}
    # The above zip is not correct; instead map properly:
    fi = {features[i]: float(mean_abs_per_feature[i]) for i in range(len(features))}
    with open(os.path.join(OUT_DIR, "shap_feature_importance.json"), "w") as f:
        json.dump(fi, f, indent=2)

    # Plot a simple bar chart of mean abs importance
    plt.figure(figsize=(6,3))
    plt.bar(features, mean_abs_per_feature)
    plt.title("Mean |SHAP value| per feature (averaged across horizon & timesteps)")
    plt.ylabel("mean(|SHAP|)")
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, "shap_feature_importance.png"))
    plt.close()

    # Save one SHAP summary image (we'll aggregate inputs by taking mean across lookback)
    # Prepare tabular background for plotting: average each sample across lookback to get (samples, features)
    tab_X = np.mean(to_explain, axis=1)  # (samples, features)
    # Build aggregated shap values similarly (average across lookback)
    shap_vals_agg = np.mean(np.stack([np.mean(arr, axis=1) for arr in shap_values], axis=0), axis=0)
    # shap_vals_agg shape (samples, features)
    shap.summary_plot(shap_vals_agg, tab_X, show=False)
    plt.savefig(os.path.join(OUT_DIR, "shap_summary_aggregated.png"))
    plt.close()
    print("SHAP explainability outputs saved to outputs folder.")
except Exception as e:
    print("SHAP DeepExplainer failed with error:", e)
    print("You can try KernelExplainer (slower) or use Tree-based models for faster SHAP.")
    with open(os.path.join(OUT_DIR,"shap_error.txt"), "w") as f:
        f.write(str(e))

# -------------------------
# 7) Save basic report and metrics summary
# -------------------------
summary = {
    "cv_metrics": metrics,
    "final_forecast_sample": pred_final.tolist()[:min(10,len(pred_final))],
    "final_model_path": os.path.join(OUT_DIR, "final_lstm_model.keras")
}
with open(os.path.join(OUT_DIR, "summary.json"), "w") as f:
    json.dump(summary, f, indent=2)

print("All outputs written to the outputs/ folder.")
print("Reference image (uploaded by you) path:", "/mnt/data/WhatsApp Image 2025-11-20 at 8.48.29 AM.jpeg")


Data generated and saved to outputs/synthetic_multivariate.csv
Supervised shapes: (1427, 60, 3) (1427, 14)
Starting walk-forward CV (this may take a few minutes)...
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Fold 0 | train_end=713 | val_end=891 | RMSE=3.1366 MAE=2.6091
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Fold 1 | train_end=891 | val_end=1069 | RMSE=2.0985 MAE=1.7162


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Fold 2 | train_end=1069 | val_end=1247 | RMSE=2.8013 MAE=2.3495


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Fold 3 | train_end=1247 | val_end=1425 | RMSE=1.7446 MAE=1.4677
Cross-validation metrics saved to outputs/cv_metrics.json
Epoch 1/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 744.4200
Epoch 2/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 253.7194
Epoch 3/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 33.8574
Epoch 4/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 28.4157
Epoch 5/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 21.6114
Epoch 6/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 11.9262
Epoch 7/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 9.1465
Epoch 8/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 7.9413
Epoch 9/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 7.8836
Epoch 10/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 7.7644
Epoch 11/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 7.5930
Epoch 12/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 6.7829
Epoch 13/30
45/45 ━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.12/dist-packages/shap/explainers/_deep/deep_tf.py:94: UserWarning: Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/keras/src/models/functional.py:241: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor_20
Received: inputs=['Tensor(shape=(200, 60, 3))']
  warnings.warn(msg)
/usr/local/lib/python3.12/dist-packages/keras/src/models/functional.py:241: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor_20
Received: inputs=['Tensor(shape=(400, 60, 3))']
  warnings.warn(msg)


SHAP DeepExplainer failed with error: in user code:

    File "/usr/local/lib/python3.12/dist-packages/shap/explainers/_deep/deep_tf.py", line 265, in grad_graph  *
        x_grad = tape.gradient(out, shap_rAnD)

    LookupError: gradient registry has no entry for: shap_TensorListStack

You can try KernelExplainer (slower) or use Tree-based models for faster SHAP.
All outputs written to the outputs/ folder.
Reference image (uploaded by you) path: /mnt/data/WhatsApp Image 2025-11-20 at 8.48.29 AM.jpeg
